In [1]:
from functions_training_pipeline import Model
import functions_training_pipeline as f

In [2]:
rf = f.load_object('RF_test_2023_04_17')

In [3]:
rf.get_attributes()

model = <class 'sklearn.ensemble._forest.RandomForestRegressor'>
hyperparameters = [{'max_depth': 4, 'n_estimators': 10}, {'max_depth': 4, 'n_estimators': 20}, {'max_depth': 7, 'n_estimators': 10}, {'max_depth': 7, 'n_estimators': 20}]
name = RF_test_2023_04_17
date_tested = 2023-04-17
columns = ['x', 'y', 'mw_value', 'col', 'row']
dates = [numpy.datetime64('2017-08-01T00:00:00.000000000'), numpy.datetime64('2017-08-02T00:00:00.000000000'), numpy.datetime64('2017-08-03T00:00:00.000000000'), numpy.datetime64('2017-08-04T00:00:00.000000000'), numpy.datetime64('2017-08-05T00:00:00.000000000'), numpy.datetime64('2017-08-06T00:00:00.000000000')]
best_hyperparameter_list = [{'max_depth': 4, 'n_estimators': 20}, {'max_depth': 4, 'n_estimators': 20}, {'max_depth': 4, 'n_estimators': 10}, {'max_depth': 4, 'n_estimators': 10}, {'max_depth': 7, 'n_estimators': 20}]
feature_importance_list = [{'x': 0.1438303719158582, 'y': 0.3450341471584463, 'mw_value': 0.12571045277256684, 'col': 0.1550098214779